In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from scipy.stats import shapiro,  poisson, chisquare, expon, kstest
from scipy import stats
from scipy.stats import norm
#import statsmodels.api as sm

# Gestión de los warnings
# -------------------------------
import warnings
warnings.filterwarnings("ignore")


###                 Fase 1: Exploracion y limpieza

##### 1. Exploración inicial.

In [ ]:
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

df= pd.read_csv('Customer_Flight_Activity.csv',header=0)
dfhist = pd.read_csv('Customer_Loyalty_History.csv',header=0)

In [ ]:
def exploracion_dataframe(dataframe):
    """
    Función que explora un dataframe.
    
    Parametros:
        dataframe (DataFrame): dataframe para explorar

    Returns:
        Esta función visualiza con prints con información 
        de los datos del dataframe
    """
    print(f"El dataframe contiene {df.shape[0]} filas y {df.shape[1]} columnas")
    print(f"De todo el conjunto de datos tenemos  {dataframe.duplicated().sum()} duplicados.")
    print("\n ..................... \n")
    display(dataframe.sample(5))
    display(dataframe.info())
    
    # generamos un DataFrame para los valores nulos
    print("De todo el conjunto de datos tenemos los siguientes nulos:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Las columnas son de tipo:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    

    print("\n ..................... \n")
    
    dataframe_categoricas = dataframe.select_dtypes(include = "O")

    print("Los valores que tenemos para las columnas categóricas son: ")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()))   
    

Exploramos el primer dataframe.

In [ ]:
exploracion_dataframe(df)

Realizamos la exploracion del segundo dataframe.

In [ ]:
exploracion_dataframe(dfhist)

Vamos a unir ambos dataframe por la columna que tienen en común, la columna Loyalty Number. Para ello vamos a usar un left join con la idea de conservar a todos los clientes, esten o no ya con nuestra compañia.

In [ ]:
df_unido = pd.merge(df, dfhist, on='Loyalty Number', how='inner')
df_unido.head(3)


Nos ha quedado un dataframe unido que inicialmente tiene la siguiente informacion:

In [ ]:
df_unido.shape

##### 2. Limpieza de datos.

Empecemos por eliminar los duplicados.Recordemos cuantos tenemos primero.

In [ ]:
df_unido.duplicated().sum()

Eliminamos y comprobamos de nuevo el tamaño.

In [ ]:
df_unido.drop_duplicates(inplace=True)
df_unido.duplicated().sum()

Antes de empezar a tratar los nulos vamos a hacer algunas limpiezas en nuestro dataframe.

In [ ]:
def limpiar_columnas(dataframe):
    """
    Limpia las columnas, las convierte en minusculas y le quita los espacios 
    entre palabras poniendoles un'_'
    
    Parametros:
    Dataframe a modificar
    
    Returns:
    Dataframe modificado.
    """
    # Convierte los nombres de las columnas a minúsculas
    dataframe.columns = [col.lower() for col in dataframe.columns]
    
   
    dataframe.columns = [col.replace(' ', '_') for col in dataframe.columns]
    #Si hay valors negativos en una columna numerica lo convertimos a nulo directamente.
    
    for col in dataframe.select_dtypes(include='number').columns:
        dataframe[col] = dataframe[col].apply(lambda x: None if x < 0 else x)
    
    
    return dataframe

In [ ]:
df_unido = limpiar_columnas(df_unido)

Vamos a empezar con los nulos. Como ya sabemos que no tiene columnas categóricas pues directamente buscamos en las numéricas.

In [ ]:
#Columnas numéricas con nulos 

nulos_num = df_unido[df_unido.columns[df_unido.isnull().any()]].select_dtypes(include = np.number).columns
print(nulos_num)

In [ ]:
# lo convertimos a DataFrame
df_nulos = pd.DataFrame((df_unido.isnull().sum() / df.shape[0]) * 100, columns = ["%_nulos"])
# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos = df_nulos[df_nulos["%_nulos"] > 0]
df_nulos




In [ ]:
df_nulos.head()

Vamos a visualizarlas primero.

Voy a empezar por las columnas de cancelaciones.Estas columnas indican si los clientes han dejado de serlo, por lo tanto los nulos los entiendo como que son clientes activos.Voy a convertir las columnas a categoricas y les voy a aplicar el valor "Activo".

In [ ]:
# # iteramos por la lista de columnas a las que le vamos a cambiar los nulos por "Uknown"
columnas = ['cancellation_year','cancellation_month']

for col in columnas:
    df_unido[col] = df_unido[col].fillna("Activo")
    df_unido[col] = df_unido[col].astype(str)
    
    # comprobamos si quedan nulos en las columnas categóricas. 
    print(f"Después del fillna' quedan los siguientes nulos {df_unido[col].isnull().sum()}")
    

In [ ]:
df_unido['cancellation_year'].unique()

In [ ]:
df_unido['cancellation_month'].unique()

Para la tabla salary, vamos a aplicarle los métodos Iterative Imputer y el KNN.
Empezamos por el Iterative Imputer.También lo compararemos con su media y con la mediana.

Empecemos mirando su media y su mediana

In [ ]:
# Mostrar estadísticas descriptivas para 'salary'
print(df_unido['salary'].describe())


Vayamos ahora con el Iterative Imputer

In [ ]:
df_unido_copy = df_unido.copy()

In [ ]:
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)
imputer_iterative_imputado = imputer_iterative.fit_transform(df_unido_copy[["salary"]])
imputer_iterative_imputado

In [ ]:
df_unido_copy["IT_salary"] = imputer_iterative_imputado
print(f"Después del 'Iterative' tenemos: \n{df_unido_copy['IT_salary'].isnull().sum()} nulos")

KNN IMPUTER

In [ ]:
# # instanciamos la clase del KNNImputer
# imputer_knn = KNNImputer(n_neighbors = 5)
# imputer_knn_imputado = imputer_knn.fit_transform(df_unido[["salary"]])
# imputer_knn_imputado

In [ ]:
#df_unido[["salary_knn"]] = imputer_knn_imputado

In [ ]:
#Comprobamos si hemos sustituido todos los nulos
#print(f"Después del 'KNN' tenemos: \n{df_unido[['salary_knn']].isnull().sum()} nulos")

In [ ]:
df_unido.columns

In [ ]:
df_unido.describe()[["salary", "salary_it"]]

In [ ]:
# ahora vamos a cambiar el nombre de las columnas que quedaron para que tengan el mismo nombre de origen
nuevo_nombre = {"salary_it": "salary"}
df_unido.rename(columns = nuevo_nombre, inplace = True)

In [ ]:
df_unido['salary'].value_counts()

In [ ]:
df_unido.columns

###                 Fase 2: Visualización

#### 1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?

In [ ]:
meses = {1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril', 5: 'mayo', 6: 'junio',
                 7: 'julio', 8: 'agosto', 9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'}

df_unido['nombre_mes'] = df_unido['month'].map(meses)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_unido, x='nombre_mes', y='flights_booked', ci=None, hue='year', palette='magma')
plt.xlabel('Mes')
plt.ylabel('Vuelos reservados')
plt.title('Distribución de Vuelos Reservados por Mes y Año')
plt.legend(title='Año')
plt.xticks(rotation=45)  
plt.show(); 


Vemos que las reservas de vuelos son mayores en 2018 que en 2017
Además apreciamos que en ambos años hay crecimiento en el número de reservas
en marzo
en mayo y sigue creciendo hata julio donde alcanza su pico y disminuye en agosto y aun más en septiembre
en diciembre
Todos estos picos corresponden con vacaciones escolares: Semana Santa, Verano y Navidad.
Apreciamos también que los meses donde se efectuan menos reservas son en enero y febrero seguidos de abril.

#### 2.¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los clientes?

In [ ]:
df_unido.columns

In [ ]:
# Crear un scatterplot
plt.figure(figsize=(7, 4))
sns.scatterplot(x="distance", y="points_accumulated", data=df_unido, color="lightcoral", alpha=0.5)
plt.title("Distancia de los vuelos en relación a los puntos acumulados")
plt.xlabel("Distancia de los vuelos")
plt.ylabel("Puntos acumulados")
plt.show()

Existe una clara tendencia positiva entre ambas variables, según sube una sube la otra, es decir, se obtienen mas puntos cuanta mas distancia se recorra.

#### 3. Cuál es la distribución de los clientes por provincia o estado?


Comprobamos y vemos que solo tenemos datos de Canada, por lo que no tendremos que movernos enytre paises, solo por provincias.

In [ ]:
df_unido['country'].unique()

In [ ]:
#Crear un countplot
plt.figure(figsize=(8, 4))
sns.countplot(x="province",data=df_unido, color="magenta")
plt.title("Distribución de clientes por provincia o estado")
plt.xlabel("Provincia o Estado")
plt.ylabel("Número de Clientes")
plt.xticks(rotation=45, ha="right")  
plt.show()


Los datos nos muestran que la mayor parte de los clientes son de Ontario, British Columbia y Quebec.
Por otro lado, las provincias con menor cantidad de clientes son Yukon y Prince Edward Island

#### 4.Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?

In [ ]:
df_unido["education"].value_counts()

In [ ]:
#rdenamos los niveles educativos
educacion_ordenada = ["High School or Below", "College", "Bachelor", "Master", "Doctor"]
educacion_ordenada

In [ ]:
df_unido['salary'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Eliminar filas con valores nulos en 'education' o 'salary'
df_unido_clean = df_unido.dropna(subset=['education', 'salary'])

# Crear el gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x='education', y='salary', data=df_unido_clean, palette='viridis', ci=None)

# Añadir etiquetas y título
plt.xlabel('Education Level')
plt.ylabel('Salary')
plt.title('Salary by Education Level')

# Rotar etiquetas del eje x para mayor legibilidad
plt.xticks(rotation=45, ha='right')

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Group by 'education' and calculate the mean salary for each group
education_salary = df_unido.groupby('education')['salary'].mean().reset_index()

# Plot the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x='education', y='salary', data=education_salary, palette='viridis', ci=None)

# Add labels and title
plt.xlabel('Education Level')
plt.ylabel('Average Salary')
plt.title('Average Salary by Education Level')

# Show the plot
plt.show()


In [ ]:
sns.set(style="whitegrid")

# Crear el gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x='education', y='salary', data=df_unido, palette='viridis', ci=None)

# Añadir etiquetas y título
plt.xlabel('Nivel Educativo')
plt.ylabel('Salario Promedio')
plt.title('Comparación del Salario Promedio por Nivel Educativo')

In [ ]:
# Definir el orden de menos a más de las values de Education
educacion_ordenada = ['High School or Below', 'College', 'Bachelor', 'Master', 'Doctor']

# Media del salario agrupado por nivel educativo
df_salario_agrupado = df.groupby('education')['salary'].mean().reindex(educacion_ordenada).reset_index()

# Crear un gráfico de barras para comparar el salario promedio por nivel educativo
plt.figure(figsize=(10, 6))
sns.barplot(x='education', y='salary', data=df_salario_agrupado, order=educacion_ordenada, palette='viridis')
plt.title('Salario Promedio por Nivel Educativo')
plt.xlabel('Nivel Educativo')
plt.ylabel('Salario Promedio')
plt.show()

In [ ]:
# Crear un grafico de barras
plt.figure(figsize=(8, 4))
sns.barplot(x="salary", y = "education" ,data=df_unido, color="darkmagenta", order=educacion_ordenada)
plt.title("Distribución del nivel educativo de los clientes y su salario")
plt.xlabel("Salario")
plt.ylabel("Nivel educativo")
plt.xticks(rotation=45, ha="right")  
plt.show()

Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?
Vemos que los clientes con doctorados son claramente los clientes con mayor salario.
Los clientes con menor nivel de estudios son los que menos cobran, seguidos de cerca por aquellos que han realizado una licenciatura.
Curiosamente cobran más los que han estudiado una diplomatura y menos los de licenciaturas.
Aquellos con un master tienen un sueldo notoriamente superior a los licenciados.
Podemos concluir que en general a mayor cantidad de estudios mayor es el salario, salvo en el caso de las diplomaturas y licenciaturas, que resulta al revés

5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?

Cómo se distribuyen los clientes según su estado civil y género?
Podemos ver que la gran mayoria de clientes, tanto en hombres como en mujeres es de casados, y los que menos viajan con nosotros son los divorciados.

#### 5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?.

In [ ]:
df_tarjetas = df_unido.groupby("loyalty_card")["loyalty_number"].count().reset_index()
df_tarjetas

In [ ]:
plt.pie("loyalty_number", labels= "loyalty_card",
        data = df_tarjetas, 
        autopct=  '%1.1f%%', 
        textprops={'fontsize': 8}, 
        startangle=90)
plt.title("Proporción de clientes - Tarjeta de fidelidad")
plt.show()


Podemos observar que la mayoría, el 45,6%, de los clientes tienen la tarjeta de fidelidad Star.
En cambio la tarjeta Aurora que tiene menor cantidad de gente es la Aurora.

In [ ]:
# Crear un grafico de barras
plt.figure(figsize=(8, 4))
sns.countplot(x="gender", hue= "marital_status" ,data=df_tarjetas, palette='dark:darkmagenta')
plt.title("Distribución los clientes según su género y su estado civil")
plt.xlabel("Genero")
plt.ylabel("Número de clientes")
plt.xticks(rotation=45, ha="right")  
plt.show()

#### 6. Cómo se distribuyen los clientes según su estado civil y género?

In [ ]:
df_unido.columns

In [ ]:


df_marital = df_unido[['marital_status', 'gender']]
plt.figure(figsize=(12, 6))
sns.countplot(x='marital_status', hue='gender', data=df_marital, palette='Set2')

# Mostrar el plot
plt.title('Distribución de Clientes según Estado Civil y Género')
plt.xlabel('Estado Civil')
plt.ylabel('Conteo')
plt.show()


Los datos nos muestran que:
El gran grueso de cliente se encuentra casado.
Los que menos viajan están divorciados.
No parece haber una diferencia de comportamiento en cuanto a género, ya que las distribuciones por estado civil son bastantes pares, así como el conteo de viajes.

### Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos que deberas seguir son:

#### 1. Preparación de Datos:

Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

In [ ]:
df_filtrado = df_unido[['flights_booked','education']]
df_filtrado.head()

#### 2.Análisis Descriptivo:

Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [ ]:
nivel_educativo = df_filtrado.groupby('education')['flights_booked'].describe()

nivel_educativo

#### 3.Prueba Estadística

Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

Para resolverlo, vamos a tomar la Hipotesis 0 H[0] y la Hipótesis 1 H[1].

H[0],será la negacion de lo que queremos demostrar, es decir, que no existe diferencia
significativa en el numero de vuelos reservados entre los clientes conniveles educativos considerados altos (Doctor,Master,Bachelor) y aquellos con niveles educativos considerados bajos(college,High Scholl or below).

La hipotesis H[1] será por el contrariio la demostración de que si existe diferencia.

In [ ]:
df_filtrado.info()

In [ ]:
df_filtrado['education_group'] = np.where(df_filtrado['education'].isin(['Doctor', 'Master', 'Bachelor']), 'Control', 'Test')

In [ ]:
df_filtrado.sample(6)

Ahora voy a hacerle un test de normalidad (Test de Shapiro) Si no tiene normalidad directamente le har'e el test de Mann-Whitney.

In [ ]:


# Dividir el conjunto de datos en dos grupos
grupo_control = df_unido[df_unido['education'].isin(['Bachelor', 'Master', 'Doctor'])]['flights_booked']
grupo_tratamiento = df_unido[df_unido['education'].isin(['College', 'High School or Below'])]['flights_booked']

# Realizar la prueba de Mann-Whitney U
estadistico_U, p_valor = stats.mannwhitneyu(grupo_control, grupo_tratamiento, alternative='two-sided')

# Definir el nivel de significancia
nivel_significancia = 0.05

# Comprobar el p-valor y tomar una decisión
if p_valor < nivel_significancia:
    print(f"Se rechaza la hipótesis nula. Existe una diferencia significativa en el número de vuelos reservados entre los grupos.")
else:
    print("No se rechaza la hipótesis nula. No hay suficiente evidencia para afirmar que hay una diferencia significativa.")
